# Parameter optimization

In [1]:
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
from glob import glob
from optimizer import Optimizer
from os import environ

# Set environment variable
environ["ENV"] = "optimize"

from config.config import ConfigFactory

pd.set_option('display.max_columns', 500)

ttype = 'buy'
pattern = ['Pattern', 'LinearReg']
indicator_list = pattern
indicator_list_higher = pattern

work_timeframe = '15m'
higher_timeframe = '1h'
opt_limit = 1000
load = False

print(f'Timeframe is {higher_timeframe}, trade type is {ttype}')

# Get configs
configs = ConfigFactory.factory(environ).configs
configs['Indicator_list'] = pattern
configs['Higher_TF_indicator_list'] = indicator_list_higher
configs['Timeframes']['work_timeframe'] = work_timeframe
configs['Timeframes']['higher_timeframe'] = higher_timeframe

optim_dict = {
                'Pattern': {'window_low_bound': [1, 2], 'window_high_bound': [3, 4, 5, 6],
                            'first_candle': [0.5, 0.6, 0.7, 0.8, 0.9], 'second_candle': [0.5, 0.6, 0.7, 0.8, 0.9]},
                'LinearReg': {'timeperiod': [3], 'low_bound': [0]}
             }

opt = Optimizer(pattern, optim_dict, **configs)
stat = opt.optimize(pattern, ttype, opt_limit, load)

# stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
# if not stat_list:
#     stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}.pkl')
# else:
#     stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}_{len(stat_list)}.pkl')


Timeframe is 1h, trade type is buy
Number of combinations is 200


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [3:35:16<00:00, 64.58s/it]


# Check local statistics

In [4]:
# 15m/1h
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']

pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('e_ratio_avg', ascending=False)
stat.head(20)

,pattern,Pattern_window_low_bound,Pattern_window_high_bound,Pattern_first_candle,Pattern_second_candle,LinearReg_timeperiod,LinearReg_low_bound,forecasts_num,e_ratio_avg,pct_price_diff_avg,e_ratio_rank,price_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
120,Pattern_LinearReg,2,3,0.9,0.5,3,0,272,1.986096,0.337500,268.218067,91.800000,1.3674,1.4563,1.6020,1.2783,2.3687,2.2438,2.2168,2.2471,2.1853,2.1856,2.1325,2.0170,1.9654,1.9517,2.0591,2.0317,2.0483,2.0509,2.0760,2.0734,2.0564,2.0222,2.0356,1.9948,0.31,0.31,0.39,0.37,0.25,0.23,0.27,0.26,0.29,0.35,0.38,0.31,0.32,0.32,0.31,0.36,0.49,0.49,0.46,0.30,0.34,0.35,0.36,0.28
20,Pattern_LinearReg,1,3,0.9,0.5,3,0,341,1.896946,0.360833,305.858529,123.044167,1.5510,1.6984,1.7591,1.4228,2.3153,2.0676,2.0465,2.0276,1.9872,1.9733,1.9378,1.8537,1.8196,1.8201,1.9199,1.8982,1.9146,1.9266,1.9523,1.9498,1.9412,1.9174,1.9255,1.9012,0.37,0.38,0.46,0.42,0.28,0.32,0.28,0.31,0.37,0.35,0.36,0.31,0.30,0.27,0.31,0.38,0.50,0.44,0.42,0.31,0.38,0.40,0.41,0.33
115,Pattern_LinearReg,2,3,0.8,0.5,3,0,456,1.774204,0.379583,353.037100,173.090000,1.3698,1.5152,1.6064,1.3638,2.0361,1.9323,1.9513,1.9717,1.9404,1.9463,1.8617,1.8031,1.7187,1.6945,1.7801,1.7720,1.7765,1.7879,1.8048,1.8002,1.7928,1.7802,1.7997,1.7754,0.32,0.36,0.42,0.38,0.29,0.28,0.30,0.29,0.33,0.37,0.39,0.37,0.42,0.35,0.38,0.40,0.49,0.52,0.53,0.36,0.43,0.45,0.36,0.32
15,Pattern_LinearReg,1,3,0.8,0.5,3,0,555,1.773496,0.396250,429.290188,219.918750,1.5513,1.7134,1.7654,1.5186,2.0581,1.8816,1.8913,1.8894,1.8750,1.8691,1.8208,1.7514,1.7102,1.6721,1.7560,1.7409,1.7508,1.7642,1.7765,1.7720,1.7653,1.7536,1.7661,1.7508,0.35,0.42,0.46,0.46,0.34,0.35,0.31,0.36,0.41,0.37,0.38,0.37,0.37,0.34,0.38,0.46,0.50,0.47,0.44,0.34,0.41,0.44,0.41,0.37
145,Pattern_LinearReg,2,4,0.9,0.5,3,0,554,1.730083,0.362083,404.466167,200.594167,1.4862,1.6348,1.7303,1.4943,2.0218,1.8665,1.8675,1.7950,1.7878,1.7746,1.7387,1.6845,1.6821,1.6834,1.7559,1.7324,1.7409,1.7344,1.7342,1.7272,1.7285,1.7058,1.7145,1.7007,0.33,0.38,0.46,0.46,0.31,0.32,0.31,0.33,0.32,0.35,0.39,0.35,0.36,0.36,0.38,0.39,0.45,0.48,0.42,0.27,0.30,0.32,0.37,0.28
195,Pattern_LinearReg,2,6,0.9,0.5,3,0,808,1.710987,0.386250,574.477900,312.090000,1.5172,1.5908,1.6900,1.5356,1.9536,1.8360,1.8295,1.7594,1.7459,1.7385,1.7002,1.6530,1.6658,1.6632,1.7200,1.7102,1.7120,1.7220,1.7199,1.7228,1.7330,1.7097,1.7169,1.7185,0.34,0.40,0.45,0.46,0.35,0.38,0.36,0.36,0.34,0.36,0.38,0.37,0.37,0.39,0.38,0.41,0.45,0.49,0.45,0.34,0.39,0.39,0.36,0.30
45,Pattern_LinearReg,1,4,0.9,0.5,3,0,621,1.707433,0.370000,439.316100,229.770000,1.5827,1.7566,1.8032,1.5606,2.0207,1.8087,1.8098,1.7280,1.7249,1.7058,1.6764,1.6295,1.6298,1.6369,1.7079,1.6867,1.6959,1.6953,1.6980,1.6916,1.6953,1.6765,1.6829,1.6747,0.35,0.42,0.49,0.50,0.33,0.37,0.32,0.37,0.36,0.36,0.37,0.34,0.32,0.33,0.37,0.41,0.47,0.45,0.37,0.27,0.31,0.32,0.37,0.31
95,Pattern_LinearReg,1,6,0.9,0.5,3,0,872,1.701475,0.396250,611.686200,345.530000,1.5994,1.6894,1.7524,1.5870,1.9625,1.7984,1.7928,1.7182,1.7099,1.6976,1.6643,1.6212,1.6347,1.6369,1.6937,1.6849,1.6861,1.6992,1.6992,1.7013,1.7127,1.6920,1.6984,1.7032,0.35,0.42,0.47,0.48,0.38,0.39,0.36,0.38,0.37,0.39,0.37,0.37,0.35,0.38,0.38,0.43,0.47,0.49,0.44,0.34,0.39,0.40,0.37,0.34
170,Pattern_LinearReg,2,5,0.9,0.5,3,0,703,1.699162,0.375833,491.511237,264.210833,1.4994,1.5779,1.6837,1.5071,1.9389,1.8250,1.8253,1.7571,1.7498,1.7413,1.

# Research parameter influence

In [22]:
param = 'Pattern_first_candle'

res = stat.groupby([param]).agg({'e_ratio_avg': 'mean',
                                 'pct_price_diff_avg': 'mean',
                                 'e_ratio_rank': 'mean', 
                                 'price_rank': 'mean', 
                                 'forecasts_num': 'sum'}).sort_values(param).reset_index()
res

,Pattern_first_candle,e_ratio_avg,pct_price_diff_avg,e_ratio_rank,price_rank,forecasts_num
0,0.5,1.487589,0.422854,660.273914,550.331146,51745
1,0.6,1.446003,0.393219,546.247849,459.464562,46048
2,0.7,1.468667,0.397260,462.588620,370.147344,36771
3,0.8,1.476269,0.409615,337.850515,271.715229,26671
4,0.9,1.456215,0.388792,207.709507,163.309125,16880


# Plot result in file

In [23]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()

x, y = param, 'e_ratio_avg'
ax.plot(res[x], res[y])
ax.set_xlabel(x)
ax.set_ylabel(y)
ax.set_title('_'.join(pattern) + '_' + ttype)

fig.savefig(f"{'_'.join(pattern)}'_'{ttype}_{param}", dpi=fig.dpi)

# Save new config data to config file

In [27]:
from config_updater import ConfigUpdater

ttype = 'buy'
pattern = ['Pattern', 'LinearReg']
work_timeframe = '15m'
higher_timeframe = '1h'
timeframe = f'{work_timeframe}_{higher_timeframe}'
        
optim_dict = {
                'Pattern': {'window_low_bound': [1], 'window_high_bound': [3],
                            'first_candle': [0.8], 'second_candle': [0.5]},
                'LinearReg': {'timeperiod': [3], 'low_bound': [0]}
             }

        
cu = ConfigUpdater(ttype, timeframe)
cu.config_update(optim_dict)